In [ ]:
import sys
sys.path.append('drive/My Drive/Colab Notebooks/KoBERT_Wellness/')
sys.path.append('drive/MyDrive/Colab Notebooks/KoBERT_Wellness/model')

In [ ]:
!pip install transformers==3.0.2
!pip install kobert-transformers==0.4.1
!pip install sentencepiece
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import random

from model.classification import KoBERTClassfication
from kobert_transformers import get_tokenizer

In [ ]:
def load_wellness_answer():
    root_path = "drive/My Drive/Colab Notebooks/KoBERT_Wellness"
    category_path = f"{root_path}/dataset/wellness_category.txt"
    answer_path = f"{root_path}/dataset/wellness_answer.txt"

    c_f = open(category_path, 'r')
    a_f = open(answer_path, 'r')

    category_lines = c_f.readlines()
    answer_lines = a_f.readlines()

    category = {}
    answer = {}
    for line_num, line_data in enumerate(category_lines):
        data = line_data.split('    ')
        category[data[1][:-1]] = data[0]

    for line_num, line_data in enumerate(answer_lines):
        data = line_data.split('    ')
        keys = answer.keys()
        if (data[0] in keys):
            answer[data[0]] += [data[1][:-1]]
        else:
            answer[data[0]] = [data[1][:-1]]

    return category, answer

In [ ]:
def kobert_input(tokenizer, str, device=None, max_seq_len=512):
    index_of_words = tokenizer.encode(str)
    token_type_ids = [0] * len(index_of_words)
    attention_mask = [1] * len(index_of_words)

    # Padding Length
    padding_length = max_seq_len - len(index_of_words)

    # Zero Padding
    index_of_words += [0] * padding_length
    token_type_ids += [0] * padding_length
    attention_mask += [0] * padding_length

    data = {
        'input_ids': torch.tensor([index_of_words]).to(device),
        'token_type_ids': torch.tensor([token_type_ids]).to(device),
        'attention_mask': torch.tensor([attention_mask]).to(device),
    }
    return data

In [ ]:
if __name__ == "__main__":
    root_path = "drive/My Drive/Colab Notebooks/KoBERT_Wellness"
    checkpoint_path = f"{root_path}/checkpoint"
    save_ckpt_path = f"{checkpoint_path}/kobert-wellnesee-classification.pth"

    # 답변과 카테고리 불러오기
    category, answer = load_wellness_answer()

    ctx = "cuda" if torch.cuda.is_available() else "cpu"
    device = torch.device(ctx)

    # 저장한 Checkpoint 불러오기
    checkpoint = torch.load(save_ckpt_path, map_location=device)

    model = KoBERTClassfication()
    model.load_state_dict(checkpoint['model_state_dict'], strict=False)

    model.to(ctx)
    model.eval()

    tokenizer = get_tokenizer()

    while 1:
        sent = input('\nQuestion: ')  # '요즘 기분이 우울한 느낌이에요'
        data = kobert_input(tokenizer, sent, device, 512)

        if '종료' in sent:
            break

        output = model(**data)

        logit = output[0]
        softmax_logit = torch.softmax(logit, dim=-1)
        softmax_logit = softmax_logit.squeeze()

        max_index = torch.argmax(softmax_logit).item()
        max_index_value = softmax_logit[torch.argmax(softmax_logit)].item()

        answer_list = answer[category[str(max_index)]]
        answer_len = len(answer_list) - 1
        answer_index = random.randint(0, answer_len)
        print(f'Answer: {answer_list[answer_index]}, index: {max_index}, category: {category[str(max_index)]}, softmax_value: {max_index_value}')
        print('-' * 50)

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Answer: 우울함은 저절로 없어지기도 하지만 그렇지 않을 때도 있어요. 그럴 때는 전문가에게 도움을 요청하는 것도 좋은 방법이에요., index: 59, category: 감정/우울감, softmax_value: 0.9568578004837036
--------------------------------------------------
Answer: 갑자기 체중이 줄어들면 몸이 힘들어 한답니다. 적당한 게 좋아요!, index: 327, category: 증상/체중감소, softmax_value: 0.08480574190616608
--------------------------------------------------
Answer: 더 울고 싶으시면 말씀하세요. 제가 손수건을 가져올게요., index: 22, category: 감정/눈물, softmax_value: 0.9508609771728516
--------------------------------------------------
Answer: 화가 많이 나셨군요. 저 같아도 그랬을 거예요. 다 이해해요., index: 81, category: 감정/화, softmax_value: 0.8127865791320801
--------------------------------------------------
Answer: 무서운 감정을 느끼셨군요. 지금은 괜찮으신가요?, index: 33, category: 감정/무서움, softmax_value: 0.8380134105682373
--------------------------------------------------
Answer: 가벼운 운동을 해보는 건 어떠세요? 산책도 좋답니다!, index: 285, category: 증상/무기력, softmax_value: 0.8374357223510742
--------------------------------------------------
Answer: 뭐든 과유불급이래요. 맛있는 음식도 너무 많